<a href="https://colab.research.google.com/github/ayuri1512/Data-Science-Projects/blob/main/price_optimization_case_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>